Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

Importing dataset

In [ ]:
data = pd.read_csv('TSLA.csv')

Training

Federated Averaging

In [ ]:
def federated_averaging(clients_data, num_rounds, learning_rate):
    global_model = LinearRegression()
    for _ in range(num_rounds):
        local_models = []
        for client_data in clients_data:
            local_model = LinearRegression()
            local_model.fit(client_data[0], client_data[1])
            local_models.append(local_model)
        global_weights = np.mean([model.coef_ for model in local_models], axis=0)
        global_model.fit(X_train, y_train)
        global_model.coef_ = (1 - learning_rate) * global_model.coef_ + learning_rate * global_weights
    return global_model

Federated Local

In [ ]:
def federated_local(clients_data, num_rounds, learning_rate):
    global_model = LinearRegression()
    for _ in range(num_rounds):
        for client_data in clients_data:
            local_model = LinearRegression()
            local_model.fit(client_data[0], client_data[1])
            global_model.fit(X_train, y_train)
            global_model.coef_ = (1 - learning_rate) * global_model.coef_ + learning_rate * local_model.coef_
    return global_model

Federated WMA

In [ ]:
def federated_wma(clients_data, num_rounds, learning_rate):
    global_model = LinearRegression()
    weights = [1 / len(clients_data)] * len(clients_data)
    for _ in range(num_rounds):
        for client_data, weight in zip(clients_data, weights):
            local_model = LinearRegression()
            local_model.fit(client_data[0], client_data[1])
            global_model.fit(X_train, y_train)
            global_model.coef_ = (1 - learning_rate) * global_model.coef_ + learning_rate * weight * local_model.coef_
    return global_model

Training

In [ ]:
X = data[['Open', 'High', 'Low', 'Volume']]
y = data['Close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
global_model = LinearRegression()

In [ ]:
clients_data = [(X_train, y_train)] * 10
num_rounds = 10
learning_rate = 0.1

fedavg_model = federated_averaging(clients_data, num_rounds, learning_rate)
fedlocal_model = federated_local(clients_data, num_rounds, learning_rate)
fedwma_model = federated_wma(clients_data, num_rounds, learning_rate)

Evaluation

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, mae, r2

In [ ]:
mse_fedavg, mae_fedavg, r2_fedavg = evaluate_model(fedavg_model, X_test, y_test)
mse_fedlocal, mae_fedlocal, r2_fedlocal = evaluate_model(fedlocal_model, X_test, y_test)
mse_fedwma, mae_fedwma, r2_fedwma = evaluate_model(fedwma_model, X_test, y_test)

Results

In [ ]:
print("Evaluation Metrics for FedAvg:")
print("MSE:", mse_fedavg)
print("MAE:", mae_fedavg)
print("R-squared:", r2_fedavg)
print()

print("Evaluation Metrics for FedLocal:")
print("MSE:", mse_fedlocal)
print("MAE:", mae_fedlocal)
print("R-squared:", r2_fedlocal)
print()

print("Evaluation Metrics for FedWMA:")
print("MSE:", mse_fedwma)
print("MAE:", mae_fedwma)
print("R-squared:", r2_fedwma)
print()

Evaluation Metrics for FedAvg:
MSE: 20.598099040233134
MAE: 1.4294596717129067
R-squared: 0.9996490747211649

Evaluation Metrics for FedLocal:
MSE: 20.59809904023323
MAE: 1.4294596717129102
R-squared: 0.9996490747211649

Evaluation Metrics for FedWMA:
MSE: 625.3184608862581
MAE: 12.26978079098647
R-squared: 0.9893465870409404

